In [ ]:
!unzip drive/MyDrive/Pets-data/images.zip
!cp -r drive/MyDrive/Pets-data/annotations .
!ls

Archive:  drive/MyDrive/Pets-data/images.zip
  inflating: images/Abyssinian_1.jpg  
  inflating: images/Abyssinian_10.jpg  
  inflating: images/Abyssinian_100.jpg  
  inflating: images/Abyssinian_100.mat  
  inflating: images/Abyssinian_101.jpg  
  inflating: images/Abyssinian_101.mat  
  inflating: images/Abyssinian_102.jpg  
  inflating: images/Abyssinian_102.mat  
  inflating: images/Abyssinian_103.jpg  
  inflating: images/Abyssinian_104.jpg  
  inflating: images/Abyssinian_105.jpg  
  inflating: images/Abyssinian_106.jpg  
  inflating: images/Abyssinian_107.jpg  
  inflating: images/Abyssinian_108.jpg  
  inflating: images/Abyssinian_109.jpg  
  inflating: images/Abyssinian_11.jpg  
  inflating: images/Abyssinian_110.jpg  
  inflating: images/Abyssinian_111.jpg  
  inflating: images/Abyssinian_112.jpg  
  inflating: images/Abyssinian_113.jpg  
  inflating: images/Abyssinian_114.jpg  
  inflating: images/Abyssinian_115.jpg  
  inflating: images/Abyssinian_116.jpg  
  inflating: ima

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import yaml
from typing import Tuple, List

import torch
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
from pydantic import BaseModel
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [72]:
# Define constants

CONFIG_FILE_PATH = 'config-mixmatch.yaml'
IMAGE_COL_IDX = 0
CLASS_ID_COL_IDX = 1
SPECIES_COL_IDX = 2
POSSIBLE_NUM_CLASSES = {2, 37}
T = 0.5
ALPHA=0.75
LAMBDA_U  = 100 / 37

In [5]:
class AdamOptimizerConfig(BaseModel):
    lr: float
    weight_decay: float

In [6]:
class Config(BaseModel):
    device: str
    num_classes: int
    batch_size: int
    max_num_epochs: int
    patience: int
    adam_optimizer_config: AdamOptimizerConfig
    num_batch_norm_layers_to_train_params: int
    num_batch_norm_layers_to_update_running_stats: int
    train_earlier_layers_delay: int
    n_hidden_layers_to_train: int
    labelled_data_ratio: float

In [78]:
with open(CONFIG_FILE_PATH, encoding='utf-8') as f:
    config_dict = yaml.load(f, Loader=yaml.FullLoader)

config = Config.model_validate(config_dict)

assert config.num_classes in POSSIBLE_NUM_CLASSES
assert 0 <= config.num_batch_norm_layers_to_train_params <= 36  # 36 batch norm layers in resnet34
assert 0 <= config.num_batch_norm_layers_to_update_running_stats <= 36  # 36 batch norm layers in resnet34

In [8]:
# K = 2
class TransformTwice(nn.Module):
    def __init__(self, transformation):
        super().__init__()
        self.transformation = transformation

    def __call__(self, data):
        return self.transformation(data), self.transformation(data)

In [9]:
class LabelledImageDataset(Dataset):
    def __init__(self, filenames: List[str], labels: List[int], use_augmentations: bool) -> None:
        self.filenames = filenames
        self.labels = labels
        # self.transformation = torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        self.transformation = (
            transforms.Compose([
                torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms(),
                # transforms.RandomCrop((what size, what other size)),
                transforms.RandomHorizontalFlip(),
                # transforms.RandomRotation((-10, 10)),
                # transforms.RandomErasing(),
            ])
            if use_augmentations
            else torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        )

    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        image = Image.open(os.path.join('images', f'{self.filenames[idx]}.jpg')).convert('RGB')
        label = self.labels[idx]

        transformed_img = self.transformation(image)

        return transformed_img.to(config.device), torch.tensor(label).to(config.device)

In [10]:
class UnlabelledImageDataset(Dataset):
    def __init__(self, filenames: List[str], use_augmentations: bool) -> None:
        self.filenames = filenames
        # self.transformation = torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        self.transformation = (
            transforms.Compose([
                torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms(),
                # transforms.RandomCrop((what size, what other size)),
                TransformTwice(transforms.RandomHorizontalFlip()),
                # transforms.RandomRotation((-10, 10)),
                # transforms.RandomErasing(),
            ])
            if use_augmentations
            else torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        )

    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        image = Image.open(os.path.join('images', f'{self.filenames[idx]}.jpg')).convert('RGB')

        transformed_img, transformed_img_2 = self.transformation(image)

        return transformed_img.to(config.device), transformed_img_2.to(config.device)

In [11]:
def get_image_names_and_labels(annotations_file_path: str, num_classes: int) -> Tuple[List[str], List[int]]:
    filenames: List[str] = []
    labels: List[int] = []

    with open(annotations_file_path, encoding='utf-8') as f:
        lines = f.readlines()

    label_col_idx = SPECIES_COL_IDX if num_classes == 2 else CLASS_ID_COL_IDX

    for line in lines:
        line_split = line.split()
        filenames.append(line_split[IMAGE_COL_IDX])
        labels.append(int(line_split[label_col_idx]) - 1)

    return filenames, labels

In [12]:
def get_batch_norm_layers(model: nn.Module) -> List[nn.Module]:
    return [
        module
        for module in model.modules()
        if isinstance(module, nn.BatchNorm2d)
    ]

In [13]:
def get_pretrained_model_with_trainable_last_layer(num_batch_norm_layers_to_train_params: int) -> nn.Module:
    model = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)

    for param in model.parameters():
        param.requires_grad = False

    batch_norm_layers = get_batch_norm_layers(model)
    batch_norm_layers_to_train_params = batch_norm_layers[-num_batch_norm_layers_to_train_params:] if num_batch_norm_layers_to_train_params else []

    for batch_norm_layer in batch_norm_layers_to_train_params:
        for param in batch_norm_layer.parameters():
            param.requires_grad = True

    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=config.num_classes)

    return model.to(config.device)

In [14]:
def make_hidden_layers_trainable(model: nn.Module, number_of_hidden_layers_to_train: int) -> nn.Module:
    trainable_layers = [layer for layer in model.modules() if not isinstance(layer, torchvision.models.resnet.ResNet) and not isinstance(layer, torchvision.models.resnet.BasicBlock) and not isinstance(layer, nn.Sequential) and not isinstance(layer, nn.Sequential) and len(list(layer.parameters())) > 0]
    trainable_layers = list(reversed(trainable_layers[:-1]))

    for l in trainable_layers[:number_of_hidden_layers_to_train]:
        for p in l.parameters():
            p.requires_grad = True

    return model

In [15]:
def get_model_accuracy(model: nn.Module, data_loader: DataLoader) -> float:
    correct_predictions_cnt = 0
    total_predictions_cnt = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Computing accuracy'):
            outputs = model(inputs)
            correct_predictions_cnt += (torch.argmax(outputs, axis=1) == labels).sum()
            total_predictions_cnt += len(outputs)
    return correct_predictions_cnt / total_predictions_cnt

In [82]:
def mix_up(X_1, X_2, Y_1, Y_2):
    assert X_1.shape == X_2.shape, 'mix_up: inputs must have same size'
    X = []
    Y = []
    for i in range(len(X_1)):
        x_1 = X_1[i]
        x_2 = X_2[i]
        y_1 = Y_1[i]
        y_2 = Y_2[i]
        l = np.random.beta(ALPHA, ALPHA)
        l = max(l, (1-l))
        x = l * x_1 + (1-l) * x_2
        y = l * y_1 + (1-l) * y_2

        X.append(x)
        Y.append(y)
    return torch.stack(X), torch.stack(Y)

In [17]:
def linear_rampup(epoch):
    return float(min(epoch / config.max_num_epochs, 100))

In [84]:
class SemiLoss(object):
    def __call__(self, outputs_x, targets_x, outputs_u, targets_u, epoch):
        probs_u = torch.softmax(outputs_u, dim=1)

        Lx = -torch.mean(torch.sum(F.log_softmax(outputs_x, dim=1) * targets_x, dim=1))
        Lu = torch.mean((probs_u - targets_u)**2)

        return Lx, Lu, LAMBDA_U * linear_rampup(epoch)

In [64]:
def infinite_batch_generator(loader: DataLoader):
    while True:
        for batch in loader:
            yield batch


def train_single_epoch(
        model: nn.Module,
        train_data_loader: DataLoader,
        unlabelled_train_data_loader: DataLoader,
        criterion: SemiLoss,
        optimizer: torch.optim.Optimizer,
        num_batch_norm_layers_to_update_running_stats: int,
        epoch: int,
        ) -> float:
    model.train()
    batch_norm_layers = get_batch_norm_layers(model)
    batch_norm_layers_to_not_update_running_stats = (
        batch_norm_layers[:-num_batch_norm_layers_to_update_running_stats]
        if num_batch_norm_layers_to_update_running_stats
        else batch_norm_layers
    )
    for batch_norm_layer in batch_norm_layers_to_not_update_running_stats:
        batch_norm_layer.eval()
    train_loss_sum = 0.0
    train_samples_cnt = 0

    unlabelled_generator = infinite_batch_generator(unlabelled_train_data_loader)

    for labelled_inputs, labelled_targets in tqdm(train_data_loader, desc='Training model'):

        labelled_targets = F.one_hot(labelled_targets, config.num_classes)
        unlabelled_inputs_1, unlabelled_inputs_2 = next(unlabelled_generator)
        unlabelled_inputs = torch.cat([unlabelled_inputs_1, unlabelled_inputs_2])

        # Calculated the pseudo label for each of the augmented unlabeled data point
        with torch.no_grad():
            u_o_1 = model(unlabelled_inputs_1)
            u_o_2 = model(unlabelled_inputs_2)
            unlabelled_targets = (F.softmax(u_o_1, dim=1) + F.softmax(u_o_2, dim=1)) / 2
            unlabelled_targets = unlabelled_targets**(1/T)
            unlabelled_targets = unlabelled_targets / unlabelled_targets.sum(dim=1, keepdim=True)

        all_inputs = torch.cat([labelled_inputs, unlabelled_inputs_1, unlabelled_inputs_2])
        all_targets = torch.cat([labelled_targets, unlabelled_targets, unlabelled_targets])
        idxs = np.random.permutation(len(all_inputs))
        all_inputs = all_inputs[idxs]
        all_targets = all_targets[idxs]

        X, targets_X = mix_up(labelled_inputs, all_inputs[:len(labelled_inputs)], labelled_targets, all_targets[:len(labelled_inputs)])
        U, targets_U = mix_up(unlabelled_inputs, all_inputs[len(labelled_inputs):], torch.cat([unlabelled_targets, unlabelled_targets]), all_targets[len(labelled_inputs):])

        outputs_X = model(X)
        outputs_U = model(U)

        LX, LU, lamb_u = criterion(outputs_X, targets_X, outputs_U, targets_U, epoch)

        optimizer.zero_grad()
        loss = LX + lamb_u * LU

        loss.backward()
        optimizer.step()
        train_loss_sum += loss.item() * (len(outputs_X) + len(outputs_U))
        train_samples_cnt += len(outputs_X) + len(outputs_U)

    return train_loss_sum / train_samples_cnt

In [20]:
def save_checkpoint(model: nn.Module, optimizer: torch.optim.Optimizer) -> Tuple[nn.Module, torch.optim.Optimizer]:
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, 'checkpoint.pt')

In [21]:
def load_checkpoint(model: nn.Module, optimizer: torch.optim.Optimizer) -> None:
    checkpoint = torch.load('checkpoint.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [69]:
filenames_trainval, labels_trainval = get_image_names_and_labels('annotations/trainval.txt', num_classes=config.num_classes)
filenames_test, labels_test = get_image_names_and_labels('annotations/test.txt', num_classes=config.num_classes)
filenames_train, filenames_val, labels_train, labels_val = train_test_split(filenames_trainval, labels_trainval, test_size=0.2, stratify=labels_trainval)

filenames_train, unlabelled_filenames_train, labels_train, _ = train_test_split(filenames_train, labels_train, train_size=config.labelled_data_ratio, stratify=labels_train)
dataset_train = LabelledImageDataset(filenames_train, labels_train, use_augmentations=True)
unlabelled_dataset_train = UnlabelledImageDataset(unlabelled_filenames_train, use_augmentations=True)

dataset_val = LabelledImageDataset(filenames_val, labels_val, use_augmentations=False)
dataset_test = LabelledImageDataset(filenames_test, labels_test, use_augmentations=False)

unlabelled_train_data_loader = DataLoader(unlabelled_dataset_train, batch_size=config.batch_size, shuffle=True)
train_data_loader = DataLoader(dataset_train, batch_size=config.batch_size, shuffle=True)
val_data_loader = DataLoader(dataset_val, batch_size=config.batch_size, shuffle=False)
test_data_loader = DataLoader(dataset_test, batch_size=config.batch_size, shuffle=False)

In [54]:
# Train using pseudo labelling
def train_model():
    model = get_pretrained_model_with_trainable_last_layer(config.num_batch_norm_layers_to_train_params)

    criterion = SemiLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.adam_optimizer_config.lr, weight_decay=config.adam_optimizer_config.weight_decay)

    max_val_accuracy = float('-inf')
    argmax_epoch = -1

    for epoch in range(config.max_num_epochs):
        print(f'Epoch #{epoch}:')
        train_loss = train_single_epoch(model, train_data_loader, unlabelled_train_data_loader, criterion, optimizer, config.num_batch_norm_layers_to_update_running_stats, epoch)
        print(f'Train loss: {train_loss}')
        val_accuracy = get_model_accuracy(model, val_data_loader)
        if val_accuracy > max_val_accuracy:
            print(f'Validation accuracy: {100 * val_accuracy:.2f}% (new best)')
            max_val_accuracy = val_accuracy
            argmax_epoch = epoch
            save_checkpoint(model, optimizer)
            print('Checkpoint saved')
        else:
            print(f'Validation accuracy: {100 * val_accuracy:.2f}% (worse than {100 * max_val_accuracy:.2f}% of epoch {argmax_epoch})')
            if epoch > argmax_epoch + config.patience:
                print(f'Early stopping')
                break
        # Start training earlier layers
        if epoch == config.train_earlier_layers_delay and config.n_hidden_layers_to_train > 0:
            print (f'Making last {config.n_hidden_layers_to_train} hidden layers trainable')
            make_hidden_layers_trainable(model, config.n_hidden_layers_to_train)
            optimizer = torch.optim.Adam(model.parameters(), lr=config.adam_optimizer_config.lr / 100, weight_decay=config.adam_optimizer_config.weight_decay)
        print()
    return model, optimizer

In [85]:
# Train the model
model, optimizer = train_model()


Epoch #0:


Training model: 100%|██████████| 5/5 [00:03<00:00,  1.33it/s]


Train loss: 3.835814155398906


Computing accuracy: 100%|██████████| 12/12 [00:04<00:00,  2.97it/s]


Validation accuracy: 0.41% (new best)
Checkpoint saved

Epoch #1:


...

Validation accuracy: 87.23% (worse than 88.04% of epoch 191)

Epoch #199:


Training model: 100%|██████████| 5/5 [00:04<00:00,  1.24it/s]


Train loss: 1.424257148511915


Computing accuracy: 100%|██████████| 12/12 [00:03<00:00,  3.04it/s]

Validation accuracy: 87.50% (worse than 88.04% of epoch 191)



In [86]:
load_checkpoint(model, optimizer)
print('Checkpoint loaded')

test_accuracy = get_model_accuracy(model, test_data_loader)

print(f'Test accuracy: {100 * test_accuracy:.2f}%')

Checkpoint loaded


Computing accuracy:  98%|█████████▊| 57/58 [00:21<00:00,  2.62it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Computing accuracy: 100%|██████████| 58/58 [00:21<00:00,  2.72it/s]

Test accuracy: 84.82%
